In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (roc_auc_score, auc, accuracy_score)
from sklearn.metrics import (roc_auc_score, auc, accuracy_score, roc_curve, confusion_matrix)
from sklearn.metrics import f1_score
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.simplefilter("ignore")
import lightgbm as lgb

/home/kazuki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
import sys
sys.path.append('../scripts')
from load_data import load_data
df = load_data()

In [4]:
# df['Sex'] = LabelEncoder().fit_transform(df['Sex'].cat.add_categories("D").fillna("D"))
df = df.drop(['Sex','Age','BMI'],axis = 1)
X_train = df.drop(['LibraryName','DiseaseState'],axis  = 1)
y_train = df['DiseaseState'].values


## メタデータを含めたlightgbmによる比較

In [13]:
n_iter = 10
folds = 3
oof = np.zeros(len(X_train))
lgb_score = []
rf_score = []

for n in range(n_iter):
    conf_mat = np.asarray([[0,0],[0,0]])
    y_probs = np.empty_like(y_train, dtype=float)
    y_trues = np.empty_like(y_train)
    y_preds = np.empty_like(y_train)
    cv_count = 0
    cv_counts = np.empty_like(y_train)
    cv = StratifiedKFold(n_splits=folds, shuffle=True)
    for train_idx, valid_idx in cv.split(X_train, y_train):
        trn_x = X_train.iloc[train_idx, :]
        val_x = X_train.iloc[valid_idx, :]
        trn_y = y_train[train_idx]
        val_y = y_train[valid_idx]
        trn_y = np.reshape(trn_y, (-1))
        val_y = np.reshape(val_y, (-1))
        
        #lightgbm
        lgb_train = lgb.Dataset(trn_x,trn_y)
        lgb_eval = lgb.Dataset(val_x,val_y,reference=lgb_train)
        lgbm_params = {
            # 多値分類問題
            'objective': 'multiclass',
            'num_class': 20,
            'learning_rate': 0.48148558790197915,
            'max_depth': 12,
            'num_leaves': 126, 
            'bagging_fraction': 0.07604494795456074,
            'feature_fraction': 0.43596480583156505, 
            'min_data_in_leaf': 137,
            'lambda_l1': 5,
            'lambda_l2': 39    
        }
        model = lgb.train(lgbm_params, lgb_train, 2000, valid_sets=[lgb_train, lgb_eval], early_stopping_rounds=100, verbose_eval=0)
        probs = model.predict(val_x)
        y_pred_max = np.argmax(probs, axis=1)
        score = f1_score(val_y,y_pred_max,average='macro')
        lgb_score.append(score)

        

print(np.mean(lgb_score))

0.4651649267547028


## メタデータなしにおけるランダムフォレストとlighgbmの違い

In [5]:
n_iter = 10
folds = 5
oof = np.zeros(len(X_train))
lgb_score = []
rf_score = []

for n in range(n_iter):
    conf_mat = np.asarray([[0,0],[0,0]])
    y_probs = np.empty_like(y_train, dtype=float)
    y_trues = np.empty_like(y_train)
    y_preds = np.empty_like(y_train)
    cv_count = 0
    cv_counts = np.empty_like(y_train)
    cv = StratifiedKFold(n_splits=folds, shuffle=True)
    for train_idx, valid_idx in cv.split(X_train, y_train):
        trn_x = X_train.iloc[train_idx, :]
        val_x = X_train.iloc[valid_idx, :]
        trn_y = y_train[train_idx]
        val_y = y_train[valid_idx]
        trn_y = np.reshape(trn_y, (-1))
        val_y = np.reshape(val_y, (-1))
        
        #lightgbm
        lgb_train = lgb.Dataset(trn_x,trn_y)
        lgb_eval = lgb.Dataset(val_x,val_y,reference=lgb_train)
        lgbm_params = {
            # 多値分類問題
            'objective': 'multiclass',
            'num_class': 20,
            'learning_rate': 0.48148558790197915,
            'max_depth': 12,
            'num_leaves': 126, 
            'bagging_fraction': 0.07604494795456074,
            'feature_fraction': 0.43596480583156505, 
            'min_data_in_leaf': 137,
            'lambda_l1': 5,
            'lambda_l2': 39    
        }
        model = lgb.train(lgbm_params, lgb_train, 2000, valid_sets=[lgb_train, lgb_eval], early_stopping_rounds=100, verbose_eval=0)
        probs = model.predict(val_x)
        y_pred_max = np.argmax(probs, axis=1)
        score = f1_score(val_y,y_pred_max,average='macro')
        lgb_score.append(score)
        
        #randomforest
        clf = RandomForestClassifier(n_estimators = 1000)
        clf.fit(trn_x, trn_y)
        probs = clf.predict(val_x)
        rf_score.append(f1_score(val_y, probs,average='macro'))

        

print(np.mean(lgb_score))
print(np.mean(rf_score))

0.4940371535815298
0.3606688702357152


In [1]:
lgb_score

NameError: name 'lgb_score' is not defined